In [1]:
from premise import *
from datapackage import Package
import bw2io as bw
import bw2data
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [2]:
ndb = NewDatabase(
    scenarios=[
        {"model":"remind", "pathway":"SSP2-NPi", "year":2050},
    ],
    source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    biosphere_name="ecoinvent-3.10-biosphere",
    keep_imports_uncertainty=True
)

premise v.(2, 3, 0, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [3]:
ndb.update("electricity")

Processing scenarios for sector 'electricity':   0%| | 0/1 [00:00<?, ?

Processing scenarios for sector 'electricity': 100%|█| 1/1 [00:17<00:0

Done!



In [4]:
db, df, cols = ndb.write_decomposition_db_to_brightway()

Dropped 144 duplicate(s).


In [5]:
# add as many columns to df as there are columns in cols
for col in cols:
    df[col] = df["original"]

In [6]:
df.columns

Index(['from activity name', 'from reference product', 'from location',
       'from categories', 'from database', 'from key', 'to activity name',
       'to reference product', 'to location', 'to categories', 'to database',
       'to key', 'flow type', 'original', 'remind - SSP2-NPi - 2050',
       'electricity - hydro - correction', 'electricity - air emissions',
       'electricity - efficiency', 'electricity - market'],
      dtype='object')

In [41]:
df.loc[df["flow type"] == "biosphere", :].head()

,from activity name,from reference product,from location,from categories,from database,from key,to activity name,to reference product,to location,to categories,to database,to key,flow type,original,remind - SSP2-NPi - 2050,electricity - hydro - correction,electricity - air emissions,electricity - efficiency,electricity - market
35,Thorium-232,None,None,"(air, non-urban air or from high stacks)",ecoinvent-3.10-biosphere,"(ecoinvent-3.10-biosphere, 4c54be62-05f1-4201-...","electricity production, at hard coal-fired pow...","electricity, high voltage",IND,None,decomposition_db_30-01-2025,"(decomposition_db_30-01-2025, 1ef94123-8873-42...",biosphere,0.0,1.296452e-06,0.0,0.0,0.0,0.0
36,"PAH, polycyclic aromatic hydrocarbons",None,None,"(air, non-urban air or from high stacks)",ecoinvent-3.10-biosphere,"(ecoinvent-3.10-biosphere, 13d898ac-b9be-4723-...","electricity production, at hard coal-fired pow...","electricity, high voltage",IND,None,decomposition_db_30-01-2025,"(decomposition_db_30-01-2025, 1ef94123-8873-42...",biosphere,0.0,7.242750e-09,0.0,0.0,0.0,0.0
38,Potassium-40,None,None,"(air, non-urban air or from high stacks)",ecoinvent-3.10-biosphere,"(ecoinvent-3.10-biosphere, 7069a2cd-257a-4a29-...","electricity production, at hard coal-fired pow...","electricity, high voltage",IND,None,decomposition_db_30-01-2025,"(decomposition_db_30-01-2025, 1ef94123-8873-42...",biosphere,0.0,1.535463e-05,0.0,0.0,0.0,0.0
39,Barium II,None,None,"(air, non-urban air or from high stacks)",ecoinvent-3.10-biosphere,"(ecoinvent-3.10-biosphere, 24cf82f8-c517-4fef-...","electricity production, at hard coal-fired pow...","electricity, high voltage",IND,None,decomposition_db_30-01-2025,"(decomposition_db_30-01-2025, 1ef94123-8873-42...",biosphere,0.0,4.135610e-08,0.0,0.0,0.0,0.0
40,Nitrogen oxides,None,None,"(air, non-urban air or from high stacks)",ecoinvent-3.10-biosphere,"(ecoinvent-3.10-biosphere, 77357947-ccc5-438e-...","electricity production, at hard coal-fired pow...","electricity, high voltage",IND,None,decomposition_db_30-01-2025,"(decomposition_db_30-01-2025, 1ef94123-8873-42...",biosphere,0.0,5.613131e-05,0.0,0.0,0.0,0.0


In [43]:
cols

['electricity - hydro - correction',
 'electricity - air emissions',
 'electricity - efficiency',
 'electricity - market']

In [46]:
import numpy as np
for act in ndb.scenarios[0]["database"]:
    for exc in act["exchanges"]:
        if any(exc.get(k, 0) != 0 for k in cols):
            #try:
            df.loc[
                (df["to activity name"] == act["name"])
                & (df["to reference product"] == act["reference product"])
                & (df["to location"] == act["location"])
                & (df["from activity name"] == exc["name"])
                & (df["from reference product"].fillna("None") == exc.get("product", "None"))
                & (df["from location"].fillna("None") == exc.get("location", "None"))
                & (df["from categories"].fillna("None") == exc.get("categories", "None"))
                & (df["flow type"] == exc["type"]),
                cols
            ] = (
                    np.array([exc.get(k, 0) for k in cols])
                    * df.loc[
                    (df["to activity name"] == act["name"])
                    & (df["to reference product"] == act["reference product"])
                    & (df["to location"] == act["location"])
                    & (df["from activity name"] == exc["name"])
                    & (df["from reference product"].fillna("None") == exc.get("product", "None"))
                    & (df["from location"].fillna("None") == exc.get("location", "None"))
                    & (df["from categories"].fillna("None") == exc.get("categories", "None"))
                    & (df["flow type"] == exc["type"]),
                    "original"
                ].values[0]
            )
            #except IndexError:
            #    print(
            #        exc
            #    )



IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
for act in ndb.database:
    for exc in act["exchanges"]:
        if "electricity - efficiency" in exc:
            print("yes")

In [6]:
ndb.database[0]["exchanges"][0].keys()

dict_keys(['name', 'product', 'unit', 'location', 'type', 'amount'])

In [10]:
ndb.scenarios[0]["database"][0]["exchanges"][2].keys()

dict_keys(['name', 'unit', 'categories', 'type', 'amount', 'input'])

In [5]:
from premise.utils import load_database
ndb.scenarios[0] = load_database(ndb.scenarios[0])

In [6]:
keys = []
for act in ndb.scenarios[0]["database"]:
    for e in act["exchanges"]:
        for k in e.keys():
            if k not in keys:
                keys.append(k)
list(set(keys))

['electricity - efficiency',
 'input',
 'Comment',
 'scale',
 'minimum',
 'simapro category',
 'production volume',
 'comment',
 'reference product',
 'ub',
 'formula',
 'categories',
 'electricity - market',
 'u2',
 'loc',
 'product',
 'allocation',
 'normalization',
 'u4',
 'system model',
 'database',
 'unit',
 'maximum',
 'name',
 'tag',
 'type',
 'original_amount',
 'simapro name',
 'dataset name',
 'shape',
 'electricity - hydro - correction',
 'u1',
 'electricity - air emissions',
 'negative',
 'u6',
 'u5',
 'u3',
 'amount',
 'input type',
 'uncertainty type',
 'pedigree',
 'location']

In [8]:
[
    k 
    for a in ndb.scenarios[0]["database"] 
    for e in a["exchanges"] 
    for k in e.keys()
    if k not in (
        'input', 'Comment', 'scale', 'minimum', 'simapro category', 
        'production volume', 'comment', 'reference product', 'ub', 
        'formula', 'categories', 'u2', 'loc', 'product', 'allocation', 
        'normalization', 'u4', 'system model', 'database', 'unit', 
        'maximum', 'name', 'tag', 'type', 'original_amount', 
        'simapro name', 'dataset name', 'shape', 'u1', 'negative', 
        'u6', 'u5', 'u3', 'amount', 'input type', 'uncertainty type', 
        'pedigree', 'location'
    )
]


['electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - air emissions',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electrici